In [75]:
# import numpy as np
# from sklearn.preprocessing import Imputer

# ratings = data['review_scores_rating']
# imp = Imputer(missing_values=np.nan, strategy='mean', axis=1)
# ratings = np.array(ratings).reshape(1, -1)
# imp.fit(ratings)
# ratings = imp.transform(ratings)
# ratings.shape
# sample_idxs = np.random.randint(ratings.shape[1], size=1000)
# sample_idxs
# np.take(ratings, sample_idxs)

In [76]:
# import math
# import matplotlib.pyplot as plt
# plt.hist(np.take(ratings, sample_idxs))
# len(data.columns)
# len(data.sample(1).values[0])
# for k, v in zip(data.columns, data.sample(1).values[0]): print k + ': ' + str(v)

In [1]:
import pandas as pd

In [2]:
all_data = pd.read_csv('./data/review_features.csv', low_memory=False)

In [251]:
data = all_data.sample(10000, random_state=1234)
cols_to_drop = ['access', 'amenities_', 'description', \
               'house_rules', 'id', 'host_verifications', 'last_review', 'last_scraped', \
               'market', 'neighborhood_overview', 'neighbourhood', 'neighbourhood_cleansed', \
               'neighbourhood_group_cleansed', 'notes', 'street', 'summary', 'transit', \
               'Unnamed: 0', 'first_review', 'calendar_last_scraped', 'calendar_updated', 
               'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', \
                'review_scores_communication', 'review_scores_location']
data.loc[:, cols_to_drop]
data = data.drop(cols_to_drop, axis=1)

data = data.replace('f', 0)
data = data.replace('t', 1)
data = data.replace(r'[\$,]','', regex=True)
# data['review_scores_rating']


# Get a list of non-numeric columns so we can drop them
from collections import defaultdict

non_numeric_columns = defaultdict(int)

for d in data.values:
    for k, v in zip(data.columns, d):
        if 'Mission' == v:
            print k, v
        
        try: 
            float(v)
        except Exception as e:
            non_numeric_columns[k] += 1

print non_numeric_columns.keys()
data = data.drop(non_numeric_columns.keys(), axis=1)

['bathrooms', 'host_has_profile_pic', 'cancellation_policy', 'host_neighbourhood', 'host_acceptance_rate', 'property_type', 'host_location', 'space', 'zipcode', 'host_since', 'host_listings_count', 'bedrooms', 'host_response_rate', 'host_about', 'calculated_host_listings_count', 'interaction', 'name', 'license', 'host_response_time', 'beds', 'cleaning_fee', 'host_name', 'guests_included', 'host_total_listings_count']


In [252]:
review_columns = defaultdict(int)
for c in data.columns:
    if 'review' in c:
        review_columns[c] += 1
review_columns.keys()

['reviews_per_month',
 'number_of_reviews',
 'review_scores_value',
 'review_scores_rating']

In [253]:
dependent_variable = 'review_scores_rating'

In [254]:
# Drop missing values from depedent variable
data = data.dropna(subset=[dependent_variable])

In [255]:
y = data[dependent_variable]
X = data.drop(review_columns.keys(), axis=1)

In [256]:
for c in X.columns:
    if 'review' in c:
        print c

In [257]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=1234)

In [258]:
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(X_train)
X_train = imp.transform(X_train)

imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(X_test)
X_test = imp.transform(X_test)

In [273]:
y_train = np.array(y_train).reshape(1, -1)
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(y_train)
y_train = imp.transform(y_train)
# y_train = y_train.reshape(-1, 1)
y_train = y_train.ravel()

y_test = np.array(y_test).reshape(1, -1)
imp = Imputer(missing_values=np.nan, strategy='median', axis=1)
imp.fit(y_test)
y_test = imp.transform(y_test)
# y_test = y_test.reshape(-1, 1)
y_test = y_test.ravel()

In [277]:
print X_train.shape
print y_train.ravel().shape

(5192, 228)
(5192,)


In [290]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, make_scorer, roc_auc_score

# Create linear regression object
regr = linear_model.Lasso()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = map(lambda x: round(x, 0), regr.predict(X_test))

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


Mean squared error: 53.97
Variance score: 0.09


In [ ]:
# MSE without dropping nan from predictor
# Mean squared error: 47.72
# Variance score: 0.08


In [285]:
for i, j in zip(y_test, y_pred): print i, j

89.0 94.0
90.0 94.0
100.0 96.0
100.0 93.0
97.0 99.0
92.0 96.0
74.0 98.0
93.0 95.0
100.0 93.0
99.0 95.0
100.0 95.0
80.0 95.0
100.0 96.0
80.0 95.0
100.0 95.0
98.0 96.0
99.0 94.0
98.0 94.0
95.0 96.0
96.0 95.0
100.0 95.0
98.0 97.0
99.0 95.0
90.0 96.0
90.0 96.0
100.0 95.0
100.0 95.0
89.0 94.0
94.0 96.0
93.0 95.0
91.0 93.0
100.0 96.0
98.0 96.0
96.0 95.0
88.0 95.0
97.0 94.0
97.0 94.0
100.0 95.0
90.0 93.0
100.0 94.0
99.0 95.0
96.0 94.0
100.0 96.0
90.0 93.0
95.0 96.0
94.0 94.0
99.0 95.0
95.0 95.0
83.0 96.0
94.0 97.0
99.0 93.0
93.0 93.0
100.0 96.0
73.0 92.0
98.0 94.0
92.0 95.0
100.0 95.0
93.0 92.0
98.0 93.0
100.0 95.0
100.0 95.0
100.0 93.0
95.0 95.0
70.0 92.0
80.0 94.0
99.0 96.0
92.0 95.0
96.0 95.0
97.0 95.0
93.0 95.0
100.0 95.0
97.0 94.0
100.0 95.0
100.0 95.0
99.0 96.0
100.0 95.0
98.0 95.0
97.0 96.0
97.0 93.0
100.0 94.0
94.0 95.0
96.0 97.0
100.0 95.0
95.0 95.0
99.0 95.0
98.0 96.0
100.0 91.0
93.0 96.0
90.0 95.0
100.0 96.0
98.0 96.0
90.0 94.0
100.0 94.0
96.0 94.0
100.0 93.0
75.0 93.0
97.0 94.0
96

100.0 94.0
96.0 94.0
97.0 94.0
97.0 93.0
100.0 94.0
94.0 94.0
100.0 93.0
97.0 94.0
96.0 95.0
93.0 96.0
83.0 95.0
100.0 93.0
74.0 91.0
93.0 94.0
100.0 95.0
97.0 94.0
100.0 92.0
91.0 94.0
100.0 94.0
80.0 96.0
97.0 98.0
100.0 96.0
93.0 96.0
100.0 94.0
96.0 96.0
87.0 96.0
100.0 96.0
89.0 94.0
80.0 92.0
98.0 95.0
98.0 93.0
83.0 95.0
100.0 95.0
96.0 93.0
90.0 94.0
99.0 95.0
100.0 96.0
90.0 94.0
100.0 93.0
93.0 96.0
85.0 92.0
90.0 96.0
92.0 94.0
92.0 94.0
100.0 95.0
92.0 92.0
99.0 96.0
92.0 97.0
85.0 93.0
83.0 93.0
96.0 93.0
100.0 95.0
88.0 96.0
92.0 95.0
97.0 93.0
100.0 95.0
87.0 95.0
92.0 96.0
100.0 95.0
100.0 95.0
93.0 95.0
100.0 93.0
100.0 94.0
99.0 96.0
100.0 95.0
100.0 95.0
91.0 95.0
76.0 95.0
89.0 94.0
100.0 95.0
93.0 95.0
87.0 95.0
99.0 94.0
93.0 96.0
96.0 93.0
99.0 95.0
91.0 95.0
99.0 93.0
92.0 95.0
94.0 94.0
95.0 94.0
100.0 93.0
94.0 95.0
100.0 95.0
98.0 95.0
98.0 94.0
100.0 95.0
87.0 93.0
94.0 95.0
98.0 97.0
100.0 97.0
80.0 95.0
95.0 93.0
100.0 95.0
100.0 93.0
100.0 95.0
100.0 95.0

100.0 96.0
98.0 95.0
93.0 95.0
100.0 92.0
96.0 95.0
85.0 102.0
92.0 95.0
80.0 93.0
90.0 93.0
95.0 94.0
95.0 94.0
94.0 94.0
100.0 94.0
95.0 94.0
96.0 94.0
87.0 95.0
80.0 93.0
96.0 95.0
100.0 95.0
95.0 95.0
85.0 95.0
93.0 94.0
100.0 95.0
92.0 94.0
88.0 93.0
95.0 93.0
100.0 96.0
87.0 92.0
91.0 96.0
96.0 96.0
95.0 94.0
81.0 92.0
90.0 96.0
100.0 93.0
97.0 95.0
75.0 94.0
96.0 95.0
82.0 94.0
90.0 92.0
100.0 95.0
91.0 94.0
93.0 95.0
87.0 94.0
93.0 93.0
93.0 95.0
98.0 96.0
93.0 93.0
90.0 94.0
95.0 93.0
96.0 95.0
99.0 95.0
100.0 95.0
99.0 95.0
100.0 95.0
93.0 93.0
77.0 94.0
100.0 94.0
97.0 94.0
100.0 95.0
100.0 93.0
60.0 93.0
90.0 96.0
99.0 94.0
100.0 91.0
98.0 93.0
95.0 95.0
97.0 94.0
100.0 93.0
100.0 94.0
95.0 92.0
85.0 94.0
94.0 95.0
100.0 96.0
94.0 95.0
98.0 95.0
93.0 95.0
99.0 96.0
96.0 95.0
96.0 93.0
92.0 95.0
99.0 94.0
100.0 93.0
91.0 95.0
80.0 95.0
96.0 93.0
100.0 95.0
96.0 93.0
100.0 95.0
91.0 94.0
98.0 95.0
85.0 95.0
97.0 95.0
100.0 93.0
89.0 94.0
100.0 93.0
96.0 93.0
98.0 95.0
100.0 9

In [ ]:
# With imputing by mean
# Mean squared error: 18099081.99
# Variance score: -371456.56

In [311]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, neural_network
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, make_scorer, roc_auc_score

# Create linear regression object
nn = neural_network.MLPRegressor(alpha=0.7, warm_start=True, activation='logistic', \
                                learning_rate='invscaling', solver='sgd', max_iter=1000)

# Train the model using the training sets
nn.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = map(lambda x: round(x, 0), nn.predict(X_test))

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))


Mean squared error: 57.26
Variance score: 0.03
